# Notebook de Preparação e Análise Exploratória dos dados


- Objetivo: Neste notebook realizamos uma simples Análise Exploratória dos dados forncecidos seguida da preparação dos arquivos necessários para carregar as imagens e seus rótulos.

Vamos clonar o repositório com o código fonte do projeto

---



In [1]:
!git clone  https://github.com/marceloluisr/Desafio_INOVAI_2025.git

Cloning into 'deteccao_pontos_chaves'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 127 (delta 1), reused 4 (delta 1), pack-reused 121 (from 1)
Receiving objects: 100% (127/127), 33.00 MiB | 27.50 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
USE_COLAB = True  # ou Falso dependendo do ambiente.

Vamos montar o drive no ambiente de execução para acessar o dados. IMPORTANTE:
a montagem do drive requer permitir acesso do Google Colab para o drive e os dados devem estar disponíveis.


In [3]:
if USE_COLAB:
  # montando google drive no ambiente para acessar dataset
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys

codigo_base_path = "/content/Desafio_INOVAI_2025/src" # [ALTERE AQUI COM O PATH DO CÓDIGO CLONADO]


# isso permite o import do módulos do código fonte no jupyter notebook
sys.path.append(codigo_base_path)
# para remover descomente  o bloco de código linha abaixo
# if codigo_base_path in sys.path:
#     sys.path.remove(codigo_base_path)

IMPORTANTE APENAS SE VOCÊ NÃO ESTIVER NO GOOGLE COLAB: antes de prosseguir, recomendo executar esse notebook dentro de um ambiente conda ou pyenv.

In [6]:
path_requisitos = "{}/requirements.txt".format(codigo_base_path)
if not USE_COLAB:
    %pip install -r {path_requisitos}
else:
    %pip install xgboost==3.0.5 mediapipe==0.10.21 rt_pose==0.0.3


AQUI2
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/lib/python3.12/importlib/metadata/__init__.py", line 819, in read_text
    return self._path.joinpath(filename).read_text(encoding='utf-8')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/pathlib.py", line 1027, in read_text
    with self.open(mode='r', encoding=encoding, errors=errors) as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/pathlib.py", line 1013, in open
    return io.open(self, mode, buffering, encoding, errors, newline)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.12/dist-packages/nvidia_cufile_cu12-1.11.1.6.dist-info/requires.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_

In [ ]:
# !pip install \
#   xgboost==3.0.5 \
#   mediapipe==0.10.21 \
#   rt_pose==0.0.3


# Imports

In [7]:
from dataset.dataset import RegressaoDataset
from utils.utils import list_image_files, salvar_lista_em_txt, carregar_lista_de_txt, salvar_dataframe, carregar_dataframe, criar_scaler_para_split
import yaml
import random
import numpy as np
import pandas as pd
import torch
import os
import zipfile
import joblib

In [8]:
## Carrega o arquivo de configuração para reproduzir os experimentos

file_path = os.path.join(codigo_base_path, "configuration.yaml")

with open(file_path, "r") as file:
    config = yaml.safe_load(file)

print(config)


{'colab': True, 'dataset_path': '/content/drive/MyDrive/Colab Notebooks/SelecaoINOVIA/datasets', 'destination_path': '/content/unzipped_files', 'log_path': './log', 'regressor_module_name': 'XGBoost', 'keypoint_module_name': 'RTPose', 'segmentation_module_name': 'U2Net', 'train': {'batch_size': 120, 'learning_rate': '1e-7', 'u2net_seg_human_model_path': ''}}


In [9]:
def set_torch_seed(seed: int = 42):
    """
    Define uma semente global para garantir reprodutibilidade em PyTorch, NumPy e random.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # para múltiplas GPUs

    # Garante comportamento determinístico (pode impactar performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    print(f"Semente definida: {seed}")
set_torch_seed(2025)

Semente definida: 2025


# Análise Exploratória e Preparação dos dados

In [10]:
diretorio_pasta_dados = config['dataset_path']

# List files
os.listdir(diretorio_pasta_dados)


['homens_15k.zip', 'medidas_dados_sinteticos.csv', 'mulheres_15k.zip']

In [ ]:
# # copia arquivo CSV do drive para o colab
# shutil.copy(diretorio_pasta_dados + '/medidas_dados_sinteticos.csv', '/content/medidas_dados_sinteticos.csv')


'/content/medidas_dados_sinteticos.csv'

O dataset contém imagens frontais e lateriais do corpo.
- cada pasta dentro do dos arquivos zip possuem nomes que indicam o ID da amostra
- Cada ID contém uma imagem frontal e outra lateral.
- 'syn_m' na subpasta indica masculino
- 'sys_f' na subpasta indica feminino
- Cada ID da imagem possui informações de medidas do corpo no arquivo medidas_dados_sinteticos.csv

In [11]:

# Pasta de destino para salvar as imagens extraídas dos arquivos compactados
# no colab, e será removido se desligar o ambiente.
destination_path = config['destination_path'] # [ALTERE AQUI se necessário]


In [12]:
## Unzip as pastas compactadas  'homens_15k.zip' e 'mulheres_15k.zip' contidas em diretorio_pasta_dados
#   para destination_path
if USE_COLAB:
  zip_file_paths = [
      diretorio_pasta_dados + '/mulheres_15k.zip',
      diretorio_pasta_dados + '/homens_15k.zip'
  ]

  os.makedirs(destination_path, exist_ok=True)

  # Unzip
  for zip_file_path in zip_file_paths:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_path)

In [13]:
## vamos observar o arquivos CSV de medidas corporais
df_medidas = pd.read_csv(os.path.join(diretorio_pasta_dados, "medidas_dados_sinteticos.csv"))
print("Total de registros: ", df_medidas.shape[0])
df_medidas.tail()


Total de registros:  460348


,id,height,chest_circ,waist_circ,hip_circ,thigh_circ,knee_circ,calf_circ,abd_circ,neck_circ,biceps_circ,split
460343,syn_m029274-2-Pos,172.49,106.04,77.40,95.89,55.96,36.90,37.62,76.82,41.59,32.89,train
460344,syn_m029274-3-Pre,172.49,109.83,72.39,96.67,56.45,36.85,39.62,70.75,42.15,35.49,unknown
460345,syn_m029274-3-Pos,172.49,109.83,72.39,96.67,56.45,36.85,39.62,70.75,42.15,35.49,unknown
460346,syn_m029275-0-Pre,189.53,97.53,90.58,107.25,60.71,39.93,37.95,95.23,39.97,27.31,unknown
460347,syn_m029275-0-Pos,189.53,97.53,90.58,107.25,60.71,39.93,37.95,95.23,39.97,27.31,unknown


In [14]:
# renomeando algumas colunas para pt-br
df_medidas.rename(columns={
    'neck_circ': 'pescoco_circ',
    'chest_circ': 'peito_circ',
    'waist_circ': 'cintura_circ',
    'hip_circ': 'quadril_circ',
    'thigh_circ': 'coxa_circ',
    'knee_circ': 'joelho_circ',
    'calf_circ': 'panturrilha_circ',
    'abd_circ': 'abdomen_circ',
    'biceps_circ': 'biceps_circ'
}, inplace=True)

df_medidas.head()

,id,height,peito_circ,cintura_circ,quadril_circ,coxa_circ,joelho_circ,panturrilha_circ,abdomen_circ,pescoco_circ,biceps_circ,split
0,syn_m000000-0-Pre,177.76,83.58,73.89,96.10,53.34,36.45,32.81,74.71,35.53,23.53,train
1,syn_m000000-0-Pos,177.76,83.58,73.89,96.10,53.34,36.45,32.81,74.71,35.53,23.53,train
2,syn_m000000-1-Pre,177.76,86.36,69.62,96.23,53.46,36.34,34.71,68.73,35.63,26.03,train
3,syn_m000000-1-Pos,177.76,86.36,69.62,96.23,53.46,36.34,34.71,68.73,35.63,26.03,train
4,syn_m000000-2-Pre,177.76,90.95,65.70,96.60,53.72,36.27,36.68,62.95,36.23,28.67,unknown


In [15]:
## comparando o total de linhas no arquivo CSV com o total de imagens

if len(os.listdir(destination_path)) * 2 != df_medidas.shape[0]:
   print("numero de registros no dataframe não é igual ao total de imagens extraídas")

numero de registros no dataframe não é igual ao total de imagens extraídas


Conforme observado na célula anterior, algumas imagens reportadas no CSV não estão contidas no pasta que extraimos com aquivos .pngs da próprias imagens. Então, vamos filtrar o Dataframe 'df_medidas' para manter apenas os IDs presentes nos arquivos de imagens.

In [16]:
## Filtrando df_medidas para manter apenas os IDs presentes nos arquivos de imagens

# Obter ids das imagens
ids_normes_arquivos = os.listdir(destination_path)
# Obter somente medidas de amostras que estão associadas com imagens
df_medidas_filtrado = df_medidas[df_medidas['id'].isin(ids_normes_arquivos)].reset_index(drop=True)

print("Antes: ", df_medidas.shape[0])
print("Depois: ", df_medidas_filtrado.shape[0])

Antes:  460348
Depois:  14806


In [17]:
df_medidas_filtrado.head()

,id,height,peito_circ,cintura_circ,quadril_circ,coxa_circ,joelho_circ,panturrilha_circ,abdomen_circ,pescoco_circ,biceps_circ,split
0,syn_m000000-2-Pos,177.76,90.95,65.70,96.60,53.72,36.27,36.68,62.95,36.23,28.67,unknown
1,syn_m000005-2-Pos,177.64,106.79,85.46,109.81,60.67,42.06,43.85,86.66,39.37,31.92,train
2,syn_m000006-2-Pre,176.09,103.48,81.51,102.64,59.55,37.98,38.66,81.92,42.12,34.49,train
3,syn_m000006-3-Pre,176.09,107.32,76.52,103.33,60.03,37.95,40.69,75.84,43.03,37.16,unknown
4,syn_m000009-3-Pre,188.28,125.20,82.22,121.38,71.56,47.27,49.72,82.69,43.27,42.62,train


## Divisão em treino, validação e teste

In [18]:
## Verificando contagem de amostras por tipo de split (train, validation, test, unknown)

df_medidas_filtrado['split'].value_counts()


,count
split,
train,7696
test,2558
val,2474
unknown,2078


In [19]:
## Agora vou particionar o Dataframe por tipo de split. Note que unknowns será considerado amostras de teste.

df_medidas_filtrado_treino = df_medidas_filtrado[df_medidas_filtrado['split'] == 'train'].reset_index(drop=True)
treino_ids = df_medidas_filtrado_treino.id.tolist()

df_medidas_filtrado_val = df_medidas_filtrado[df_medidas_filtrado['split'] == 'val'].reset_index(drop=True)
val_ids = df_medidas_filtrado_val.id.tolist()


df_medidas_filtrado_teste = df_medidas_filtrado[df_medidas_filtrado['split'].isin(['test', 'unknown'])].reset_index(drop=True)
teste_ids = df_medidas_filtrado_teste.id.tolist()



In [20]:
df_medidas_filtrado_treino.head(1)

,id,height,peito_circ,cintura_circ,quadril_circ,coxa_circ,joelho_circ,panturrilha_circ,abdomen_circ,pescoco_circ,biceps_circ,split
0,syn_m000005-2-Pos,177.64,106.79,85.46,109.81,60.67,42.06,43.85,86.66,39.37,31.92,train


Como o dataframe originado do arquivo CSV somente possui informações associadas com um ID único para cada amostra e sabendo que cada amostra possui duas imagens para a parte frontal e lateral do corpo, então vou obter o caminho completo de cada imagem associada com o ID por tipo de split.

In [21]:
diretorios_imagens = list_image_files(destination_path)

## Filtrando os caminhos cujas subpastas estão em train_ids
treino_diretorios_imagens = [
    os.path.relpath(path, config['destination_path']) for path in diretorios_imagens
    if os.path.basename(os.path.dirname(path)) in treino_ids
]
## Filtrando os caminhos cujas subpastas estão em val_ids
validacao_diretorios_imagens = [
    os.path.relpath(path, config['destination_path']) for path in diretorios_imagens
    if os.path.basename(os.path.dirname(path)) in val_ids
]

## Filtrando os caminhos cujas subpastas estão em teste_ids
teste_diretorios_imagens = [
    os.path.relpath(path, config['destination_path']) for path in diretorios_imagens
    if os.path.basename(os.path.dirname(path)) in teste_ids
]

print("Total de imagens para treino: ", len(treino_diretorios_imagens))
print("Total de imagens para validação: ", len(validacao_diretorios_imagens))
print("Total de imagens para teste: ", len(teste_diretorios_imagens))

Total de imagens para treino:  15392
Total de imagens para validação:  4948
Total de imagens para teste:  9272


Observe que o total de imagens por split é justamente o dobro de amostras no dataframe associado com aquele split pois temos ortientação front e left para cada ID de amostra coletada.

Agora que possuimos os caminho completos das imagens e informações associadas com cada imagem, então podemos prosseguir com a criação do Dataset Pytorch.

In [22]:
config['log_path'] = "/content/deteccao_pontos_chaves/log"

In [23]:
# vamos salvar os dados dos caminhos das imagens e split de datframe em arquivos de log para futura recuperação
salvar_lista_em_txt(
    lista_caminhos = treino_diretorios_imagens,
    caminho_arquivo = os.path.join(config['log_path'], "treino_diretorios_imagens.txt"))

salvar_lista_em_txt(
    lista_caminhos = validacao_diretorios_imagens,
    caminho_arquivo = os.path.join(config['log_path'], "validacao_diretorios_imagens.txt"))

salvar_lista_em_txt(
    lista_caminhos = teste_diretorios_imagens,
    caminho_arquivo = os.path.join(config['log_path'], "teste_diretorios_imagens.txt"))

salvar_dataframe(
    df = df_medidas_filtrado_treino,
    caminho_arquivo = os.path.join(config['log_path'], "df_medidas_filtrado_treino.csv")
)


salvar_dataframe(
    df = df_medidas_filtrado_val,
    caminho_arquivo = os.path.join(config['log_path'], "df_medidas_filtrado_val.csv")
)


salvar_dataframe(
    df = df_medidas_filtrado_teste,
    caminho_arquivo = os.path.join(config['log_path'], "df_medidas_filtrado_teste.csv")
)



In [24]:
# testando carregamento
treino_diretorios_imagens = carregar_lista_de_txt(os.path.join(config['log_path'], "treino_diretorios_imagens.txt") )
df_medidas_filtrado_treino = carregar_dataframe(os.path.join(config['log_path'], "df_medidas_filtrado_treino.csv") )

validacao_diretorios_imagens = carregar_lista_de_txt(os.path.join(config['log_path'], "validacao_diretorios_imagens.txt") )
df_medidas_filtrado_val = carregar_dataframe(os.path.join(config['log_path'], "df_medidas_filtrado_val.csv") )


teste_diretorios_imagens = carregar_lista_de_txt(os.path.join(config['log_path'], "teste_diretorios_imagens.txt") )
df_medidas_filtrado_teste = carregar_dataframe(os.path.join(config['log_path'], "df_medidas_filtrado_teste.csv") )

## Dataset Pytorch

- Vamos testar a criação do /dataset/RegressaoDataset

A célula abaixo realiza a criação de um *scaler* para efetura Z-Score Normalization para o split de treino. Essa normalização vai
ser útil para padronizar os vetores de características multivaridos que devemos estimar por
regressão. A motivação é que Funções de Perda comuns, como MSE (Erro Quadrático Médio), são sensíveis à magnitude. Valores maiores dominam a perda,
mesmo que sejam menos significativos. Essa normalização do vetor de característica é inspirada de uma pratica comum em Séries Temporais multivaridadas.


In [25]:


criar_scaler_para_split(
    dataframe=df_medidas_filtrado_treino,
    dir_arquivo_saida=os.path.join(config['log_path'], "treino_scaler.pkl")
)


Observe abaixo que apliquei o scaler somente no treino, pois a validação é uma simulação do teste. O scaler possui  método 'inverse_transform' responsável por transformar as valores padronizados para o espaço dos valores reais das medidas. Logo, durante a inferência vou usar este scaler para reverter a saída para a escala dos dados originais a fim de permitir a comparação.

In [26]:

treino_scaler = joblib.load(os.path.join(config['log_path'], "treino_scaler.pkl"))


treino_dataset = RegressaoDataset(
    image_paths = treino_diretorios_imagens,
    scaler = treino_scaler,
    dataframe = df_medidas_filtrado_treino,
    dataset_path = config['destination_path'],
    u2net = True if config['segmentation_module_name'] == 'U2Net' else False,
)



val_dataset = RegressaoDataset(
    image_paths = validacao_diretorios_imagens,
    scaler = None,
    dataframe = df_medidas_filtrado_val,
    dataset_path = config['destination_path'],
    u2net = True if config['segmentation_module_name'] == 'U2Net' else False,
)



teste_dataset = RegressaoDataset(
    image_paths = teste_diretorios_imagens,
    scaler = None,
    dataframe = df_medidas_filtrado_teste,
    dataset_path = config['destination_path'],
    u2net = True if config['segmentation_module_name'] == 'U2Net' else False,
)



# Próximos passos


Agora que salvamos os arquivos de carregamentos dos dados na pasta config['log_path'], então podemos prosseguir para o próximo notebook responsável por Modelagem e Treinamento.